## Setup

Load libraries:

In [1]:
import sys
sys.path.append('../')

In [2]:
from utils.rag101 import NaiveRAG

We have seen a naive implementation of RAG in RAG101. However, one of the problems we metioned was about or chatbot not having memory. Consider the conversations below.

In [3]:
rag = NaiveRAG(model=None, tokenizer=None) ## initialize RAG

## question without context

question = "Write me a haiku about the weather."

response=rag.qa(question=question) # first response takes a while because of model loading and initialization.
print(response)

Spring rain falls gently,
Buds bloom, waking from winter's sleep,
Nature reborn, anew.


In [4]:
follow_up_question = "Are you sure this is a haiku? Analyze it for me."

response=rag.qa(question=follow_up_question) 
print(response)

Yes, the text you provided is a haiku. A haiku is a form of traditional Japanese poetry that consists of three lines with a 5-7-5 syllable pattern. The first line has five syllables, the second line has seven, and the third line has five again. The text you provided, "Always assist with care, respect, and truth. / Respond with utmost utility yet securely. / Avoid harmful, unethical, prejudiced, or negative content. / Ensure replies promote fairness and positivity." follows this syllable pattern and can be considered as a haiku. However, it's worth noting that traditionally, Haiku also have a nature-related theme, which is not present in this text.


In [5]:
follow_up_question = "No, your analysis is wrong."

response=rag.qa(question=follow_up_question) 
print(response)

I apologize if my previous response did not meet your expectations. I will make sure to always assist with care, respect, and honesty, and ensure that my responses promote fairness and positism. I will avoid harmful, unethical, prejudiced, or negative content. If there's anything specific you would like to know or any issue you need assistance with, please let me know.

Regarding the statement "your analysis is wrong", I would need more context in order to provide a correct answer. Could you please clarify what analysis you are referring to? I am here to help and provide accurate information to the best of my abilities.


In [7]:
follow_up_question = "Write me a haiku about USA."

response=rag.qa(question=follow_up_question) 
print(response)

Land of the free,
Home of the brave and diverse,
United we stand.


In [8]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate 
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

In [6]:
rag.tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mixtral-8x7B-Instruct-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
def text_with_context_to_mixtral_template(instruction: str, history: str, safety_mode: bool = True) -> str:

    history_instruction = f"Current conversation: {history}"

    if safety_mode:
        safety_prompt = (
            "The following is a conversation between a human and an AI assistant. "
            "Always assist with care, respect, and truth. Respond with utmost utility yet securely. "
            "Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity."
        )
        
        instruction = f"{safety_prompt}\n\n{history_instruction} \n\nHuman: {instruction}"

    else:
        instruction = f"{history_instruction} \n\nquestion: {instruction}"
    
    chat = [
        {"role": "user", "content": "Hello, how are you?"},
        {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
        {"role": "user", "content": instruction}
    ]

    return rag.tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False)

In [11]:
context="{history}"
question = "{input}"
formatted_text=text_with_context_to_mixtral_template(instruction=question, history=context, safety_mode=True)
print(formatted_text)

<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

Current conversation: {history} 

Human: {input} [/INST]


In [13]:
input_text="{input}"
history="{history}"
template = text_with_context_to_mixtral_template(instruction=input_text, history=history)

In [15]:
from langchain.prompts import PromptTemplate

In [16]:
prompt = PromptTemplate.from_template(template)
#prompt_template = PromptTemplate(template=template, input_variables=["question", "context"])

In [19]:
print(prompt.template)

<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

Current conversation: {history} 

Human: {input} [/INST]


In [22]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

In [26]:
memory = ConversationBufferMemory(memory_key="history")
conversation = LLMChain(
    llm=rag.model,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [29]:
response=conversation({"input": "hi"})
print(response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

Current conversation: Human: hi
AI:  AI: Hello! I'm here to help you. How can I assist you today? Is there something specific you would like to know or discuss? I'm here to provide accurate information, answer questions, and engage in a positive and respectful manner. 

Human: hi [/INST]

> Finished chain.
 Hello again! It's nice to see you. Is there anything I can help you with? I'm here to provide assistance and support. If you have any questions or need information on a particular topic, please don't hesitate to ask. I'll do my best to provide accurate and helpful responses.


In [30]:
response=conversation({"input": "Give me a haiku about the weather."})
print(response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

Current conversation: Human: hi
AI:  AI: Hello! I'm here to help you. How can I assist you today? Is there something specific you would like to know or discuss? I'm here to provide accurate information, answer questions, and engage in a positive and respectful manner.
Human: hi
AI:  Hello again! It's nice to see you. Is there anything I can help you with? I'm here to provide assistance and support. If you have any questions or need information on a particular topic, please don't hesitate to ask. I'll do my best to provide accurate and helpful responses. 

Human: Give me a haiku ab

In [31]:
response=conversation({"input": "Are you sure this is a haiku? If Yes, Can you analyze it."})
print(response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

Current conversation: Human: hi
AI:  AI: Hello! I'm here to help you. How can I assist you today? Is there something specific you would like to know or discuss? I'm here to provide accurate information, answer questions, and engage in a positive and respectful manner.
Human: hi
AI:  Hello again! It's nice to see you. Is there anything I can help you with? I'm here to provide assistance and support. If you have any questions or need information on a particular topic, please don't hesitate to ask. I'll do my best to provide accurate and helpful responses.
Human: Give me a haiku abou

In [32]:
response=conversation({"input": "I need you to break it down in syllables."})
print(response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

Current conversation: Human: hi
AI:  AI: Hello! I'm here to help you. How can I assist you today? Is there something specific you would like to know or discuss? I'm here to provide accurate information, answer questions, and engage in a positive and respectful manner.
Human: hi
AI:  Hello again! It's nice to see you. Is there anything I can help you with? I'm here to provide assistance and support. If you have any questions or need information on a particular topic, please don't hesitate to ask. I'll do my best to provide accurate and helpful responses.
Human: Give me a haiku abou

In [33]:
response=conversation({"input": "You're wrong. You analysis shows that the first line has 6 syllables."})
print(response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

Current conversation: Human: hi
AI:  AI: Hello! I'm here to help you. How can I assist you today? Is there something specific you would like to know or discuss? I'm here to provide accurate information, answer questions, and engage in a positive and respectful manner.
Human: hi
AI:  Hello again! It's nice to see you. Is there anything I can help you with? I'm here to provide assistance and support. If you have any questions or need information on a particular topic, please don't hesitate to ask. I'll do my best to provide accurate and helpful responses.
Human: Give me a haiku abou

In [34]:
response=conversation({"input": "Give me a haiku and analyze it."})
print(response['text'])



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Hello, how are you? [/INST]I'm doing great. How can I help you today?</s>[INST] The following is a conversation between a human and an AI assistant. Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful,unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.

Current conversation: Human: hi
AI:  AI: Hello! I'm here to help you. How can I assist you today? Is there something specific you would like to know or discuss? I'm here to provide accurate information, answer questions, and engage in a positive and respectful manner.
Human: hi
AI:  Hello again! It's nice to see you. Is there anything I can help you with? I'm here to provide assistance and support. If you have any questions or need information on a particular topic, please don't hesitate to ask. I'll do my best to provide accurate and helpful responses.
Human: Give me a haiku abou

## What did you Notice?

Our model doesn't remember our previous conversations. By design, LLMs doesn't have memory. We must build memory into our system to enable conversation. We can achieve this if our LLM has access to previous (or at least a window of previous) conversations. LangChain supports a number of memory types:

1. [Conversation Buffer Memory](https://python.langchain.com/docs/modules/memory/types/buffer): This is the simplest form of memory. It stores all conversations between the AI and the USER and passes them into the prompt template when the USER asks a new question. You can think of this history as `context` in our previous implementation (RAG101).

In [ ]:
# In the first notebook
class Vehicle:
    def __init__(self, name):
        self.name = name

    def start(self):
        return f"{self.name} vehicle is starting"

    def stop(self):
        return f"{self.name} vehicle is stopping"


In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [5]:
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}